In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee as mc
from scipy import stats

%matplotlib inline

In [18]:
def createMockCluster(rin,rout,nmean,k,smax,m,vmean,threeD=False):
    ntot = np.random.poisson(lam=nmean)
    
    nu0 = ntot/(4*np.pi*(np.log(rout/rin))) if k==3. else ntot*(3.-k)/(4*np.pi*(rout**(3.-k)-rin**(3.-k)))
    #rho = 4*np.pi*nu0*np.random.uniform(low=rin,high=rout,size=ntot)**(2.-k)
    rho = np.power(np.random.uniform(4*np.pi*nu0*rin**(2.-k),4*np.pi*nu0*rout**(2.-k),size=ntot)/(4*np.pi*nu0),1./(2-k))
    phi = np.random.uniform(0.,2*np.pi,ntot)
    theta = np.arccos(np.random.uniform(-1,1,ntot))
    
    x = rho*np.sin(theta)*np.cos(phi)
    y = rho*np.sin(theta)*np.sin(phi)
    
    vobs = np.array([np.random.normal(loc=0.,scale=(1/np.sqrt(3))*smax/(i**m))+vmean for i in rho])
    
    if threeD:
        z = rho*np.cos(theta)
        return pd.DataFrame(np.hstack((x.reshape((ntot,1)),y.reshape((ntot,1)),z.reshape((ntot,1)),\
                                      vobs.reshape((ntot,1)),rho.reshape((ntot,1)))),columns=['x','y','z','vobs','rho'])
    else:
        r = np.sqrt(np.power(x,2.)+np.power(y,2.))
        return pd.DataFrame(np.hstack((r.reshape((ntot,1)),vobs.reshape((ntot,1)))),columns=['r','vobs'])

In [19]:
#load data

data=pd.read_csv('continuousMock5.csv')
#data.rename(columns={'Unnamed: 0':'id','0':'origAnn','1':'x','2':'y','3':'z','4':'vobs'},inplace=True)
#
#data['r'] = np.power(np.power(data['x'],2.)+np.power(data['y'],2.),0.5)

def defineAnnuli(r,nAnn):
    return np.append(0.,np.sort(r)[np.linspace(0,len(r)-1,nAnn).astype(int)][1:])

nAnn = 10
annuli = defineAnnuli(data['r'],nAnn)

print annuli

[   0.            0.95736622    1.20732841    1.57089614    2.17402307
    3.21106781    5.90959364   11.07235251   33.2303394   755.40312831]


In [20]:
def labelGalaxies(data,annuli):
    #adds a column to data indicating the galaxy's membership in an annulus
    bla = np.zeros(len(data['r']))
    for i in range(len(annuli)-1):
        bla += i*np.array(np.logical_and(data['r']>annuli[i],data['r']<=annuli[i+1]).astype(int))
    return bla

def galInAnn(data,annuli):
    #calculates the number of galaxies in all the annuli
    return np.array([(data['annuli']==i).sum() for i in range(len(annuli)-1)])

def sigInAnn(data,annuli):
    return [np.std(data['vobs'][data['annuli']==i],ddof=1) for i in range(len(annuli)-1)]

def getProfiles(annuli,rin,rout,nmean,k,smax,m,vmean):
    mock = createMockCluster(rin,rout,nmean,k,smax,m,vmean)
    mock['annuli'] = labelGalaxies(mock,annuli)
    return galInAnn(mock,annuli),sigInAnn(mock,annuli)

In [21]:
data['annuli'] = labelGalaxies(data,annuli)

nObs = galInAnn(data,annuli)
sObs = sigInAnn(data,annuli)

vMean = np.mean(data['vobs'])

print nObs,sObs

[112 111 111 111 111 111 111 111 111] [1531.9828886951695, 1512.731745345587, 1311.3786694314058, 1017.1638266071807, 1008.78546367375, 756.1698132039214, 578.9581236078482, 389.46051704842796, 177.43934802582393]


In [22]:
#all log likelihoods

#make a table of log(n!)-s for n=1,...
#factorialTable = np.cumsum(np.append(0,[np.log(i) for i in range(1,len(data['r']))]))

def logPriorK(k):
    #starting gaussian
    #nu0 = 3. #mean prior density index
    #sigma = 1.
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorM(m):
    #starting gaussian
    #nu0 = -.2 #mean prior sigma index
    #sigma = 0.1
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorSigma0(sigma0):
    #starting gaussian
    #nu0 = 1000 #mean prior sigma index
    #sigma = 500
    if sigma0<10.:
        return -1e20
    else:
        return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorNmean(nmean):
    if nmean<0.: 
        return -1e20
    else:
        return 0.0

def logPriorVmean(vmean):
    return 0.0

def logLikelihoodNu(prediction,data):
    return np.sum(data*np.log(prediction)-prediction)

def logLikelihoodSigma(vObs,vPred,nObs):
    # distribution of variances is chisq, as shown, for example, at
    # https://onlinecourses.science.psu.edu/stat414/node/174
    return np.sum(np.log(stats.chi2.pdf((nObs-1.)*np.power(vObs,2.)/np.power(vPred,2.),nObs-1)))

def logPosterior(params,annuli,rin,rout,nObs,sObs):
    nmean,k,smax,m,vmean = params
    if nmean<0. or smax<0.: return -np.inf
    nPred,sPred = getProfiles(annuli,rin,rout,50*nmean,k,smax,m,vmean)
    if not np.all(nPred>=3.): return -np.inf
    nPred/=50
    return logPriorK(k)+logPriorM(m)+logPriorSigma0(smax)+logPriorNmean(nmean)+logPriorVmean(vmean)+\
           logLikelihoodNu(nPred,nObs)+logLikelihoodSigma(sObs,sPred,nObs)

In [23]:
nmean,k,smax,m,vmean=[1.30539316e+03,1.19789549e+00,1.36928408e+03,1.91498109e-01,-1.73709495e+02]
#print logPosterior(pars,annuli,1,1000,nObs,sObs)

nPred,sPred = getProfiles(annuli,1,1000,50*1.30539316e+03,1.19789549e+00,1.36928408e+03,1.91498109e-01,-1.73709495e+02)
nPred/=50.
print nPred,sPred
#print nPred,sPred
print logLikelihoodSigma(sObs,sPred,nObs)#+logLikelihoodNu(nPred,nObs)+0.+0.+0.+0.

[ 124    1    2    3    4   11   20   69 1065] [230.44628925927174, 742.3473793120052, 618.339076900354, 600.3354236322857, 624.2374261832506, 568.105560912394, 469.71694077493123, 402.1793906133739, 264.23404452963445]
-inf


/home/ondrej/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.


In [24]:
ntot=1000
bla1,bla2 = getProfiles(annuli,1,1000,50*ntot,3,1000,0.2,0)

bla1/=50

print bla1
print bla2

[190 159 151 139 114 111  64  47  22]
[547.831490919255, 547.7297745835538, 520.2175416672916, 496.05669513941757, 458.5913662004728, 415.90735689004134, 365.06224513866584, 313.897496998565, 240.25056999206762]


In [25]:
nwalkers = 40
ndim = 5

#initial = np.tile(np.array([np.random.normal(1000,100),np.random.normal(3,0.3),np.random.normal(1000,100),np.random.normal(0.2,0.02)]),10).reshape((nwalkers,ndim))
initial = np.array([np.random.normal(1000,150),np.random.normal(3.,0.3),\
                    np.random.normal(1000,100),np.random.normal(0.2,0.03),\
                    np.random.normal(0,100)])

for i in range(nwalkers-1):
    initial=np.vstack((initial,np.array([np.random.normal(1000,150),np.random.normal(3.,0.3),\
                    np.random.normal(1000,100),np.random.normal(0.2,0.03),\
                                         np.random.normal(0,100)])))

#for i in range(nwalkers):
#    for j in range(ndim):
#        initial[i,j]+=np.random.normal()


In [26]:
rin = 1
rout = 1000
sampler = mc.EnsembleSampler(nwalkers, ndim, logPosterior, args=[annuli,rin,rout,nObs,sObs],threads=4)

#p0 = np.array([1000,3,1000,0.2])reshape((nwalkers,ndim))

#pos, prob, state = sampler.run_mcmc(initial, 5)


In [27]:
pos, prob, state = sampler.run_mcmc(initial, 50)
sampler.reset()

KeyboardInterrupt: 

In [ ]:
#sampler.reset()
for i in range(75):
    pos, prob, state = sampler.run_mcmc(pos, 5)
    print "Step "+str(50*(i+1))

In [ ]:
#chain = np.load('chain_conMock0_15000steps.npy')

print sampler.flatchain.shape

import corner
corner.corner(sampler.flatchain, labels=['ntot','k','sigma0','m','vmean'],quantiles=[0.16,0.5,0.84],
                show_titles=True, title_args={"fontsize": 12},
                plot_datapoints=True, fill_contours=True, levels=[0.68, 0.95], color='b', bins=40, 
                smooth=1.0);
plt.show()

In [ ]:
chain = np.load('chain_virgoDataMembers_10500steps.npy')

#print sampler.flatchain.shape

import corner
corner.corner(chain, labels=['ntot','k','sigma0','m','vmean'],quantiles=[0.16,0.5,0.84],
                show_titles=True, title_args={"fontsize": 12},
                plot_datapoints=True, fill_contours=True, levels=[0.68, 0.95], 
                color='b', bins=40, smooth=1.0);
plt.show()

In [ ]:
radii = np.logspace(0,3,20)

def generateMassProfile(radii,sigma0,m,k):
    return -radii*(sigma0*radii**(-m))**2*(-k-2*m)

for i in sampler.flatchain[14000:]:
    plt.plot(radii,generateMassProfile(radii,i[2],i[3],i[1]),alpha=0.01)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('radius')
plt.ylabel('mass (arb.units)')
    

In [ ]:
np.save('chain_continuousMock5_15000steps.npy',sampler.flatchain)